# Desafio Técnico - Cientista de Dados Júnior
### Arthur Gusmão

## Localização de chamados do 1746

In [80]:
## imports
import pandas as pd
import basedosdados as bd
import numpy as np

In [18]:
## queries

query = """
SELECT *
FROM `datario.administracao_servicos_publicos.chamado_1746`
WHERE DATE(data_inicio) = '2023-04-01';
"""

query_bairros = """
SELECT *
FROM `datario.dados_mestres.bairro`
"""

query_5071 = """
SELECT *
FROM `datario.administracao_servicos_publicos.chamado_1746` 
WHERE DATE(data_inicio) BETWEEN '2022-01-01' AND '2023-12-31' AND id_subtipo = '5071';
"""

query_eventos = """
SELECT *
FROM `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`
"""

In [114]:
df = bd.read_sql(query, billing_project_id="arthur-data-engineering-course",)
df_bairros = bd.read_sql(query_bairros, billing_project_id="arthur-data-engineering-course",)


Downloading: 100%|██████████|
Downloading: 100%|██████████|


### Questão 1:
#### Quantos chamados foram abertos no dia 01/04/2023?

In [8]:
df['id_chamado'].nunique()


73

### Questão 2:
#### Qual o tipo de chamado que teve mais teve chamados abertos no dia 01/04/2023?

In [10]:
df \
    .groupby(['id_tipo', 'tipo'])['id_chamado'].nunique() \
    .reset_index(name='total_chamados') \
    .sort_values(by='total_chamados', ascending=False) \
    .head(1)

,id_tipo,tipo,total_chamados
2,1393,Poluição sonora,24


### Questão 3:
#### Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?

In [14]:
pd.merge(df,
        df_bairros,
        left_on='id_bairro',
        right_on='id_bairro',
        how='left') \
    .groupby('nome')['id_chamado'].nunique() \
    .reset_index(name='total_chamados') \
    .sort_values(by='total_chamados', ascending=False) \
    .head(3)

,nome,total_chamados
10,Engenho de Dentro,8
3,Campo Grande,6
20,Leblon,6


### Questão 4
#### Qual o nome da subprefeitura com mais chamados abertos nesse dia?

In [15]:
pd.merge(df,
        df_bairros,
        left_on='id_bairro',
        right_on='id_bairro',
        how='left') \
    .groupby('subprefeitura')['id_chamado'].nunique() \
    .reset_index(name='total_chamados') \
    .sort_values(by='total_chamados', ascending=False) \
    .head(1)

,subprefeitura,total_chamados
5,Zona Norte,25


### Questão 5:
#### Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?

In [16]:
pd.merge(df,
        df_bairros,
        left_on='id_bairro',
        right_on='id_bairro',
        how='left') \
    .loc[(df['id_bairro'].isnull()) | (df_bairros['subprefeitura'].isnull()), ['id_chamado', 'tipo', 'subtipo']]

# Sim, o identificado por 18516246. Por ser um chamado de ônibus, para a verificação do funcionamento do ar condicionado, é dificil precisar a localização do ocorrido.

,id_chamado,tipo,subtipo
0,18516246,Ônibus,Verificação de ar condicionado inoperante no ô...


## Chamados do 1746 em grandes eventos

### Questão 6:
#### Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?

In [67]:
df_5071 = bd.read_sql(query_5071, billing_project_id="arthur-data-engineering-course",)

Downloading: 100%|██████████|


In [20]:
df_5071['id_chamado'].nunique()

42408

### Questão 7:
#### Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).

In [65]:
df_eventos = bd.read_sql(query_eventos, billing_project_id="arthur-data-engineering-course",)

Downloading: 100%|██████████|


In [81]:
df_5071['evento'] = np.nan

for index, row in df_5071.iterrows():
    data_inicio = row['data_inicio'].date()
    for _, evento_row in df_eventos.iterrows():
        if evento_row["data_inicial"] <= data_inicio <= evento_row["data_final"]:
            df_5071.at[index, 'evento'] = evento_row["evento"]
            break


In [86]:
df_5071[df_5071['evento'].notnull() ]

,id_chamado,data_inicio,data_fim,id_bairro,id_territorialidade,id_logradouro,numero_logradouro,id_unidade_organizacional,nome_unidade_organizacional,id_unidade_organizacional_mae,...,prazo_unidade,prazo_tipo,dentro_prazo,situacao,tipo_situacao,justificativa_status,reclamacoes,data_particao,geometry,evento
7217,18078140,2022-12-30 20:18:43,2022-12-30 22:03:07,23,2,65615,93,70,GM-RIO - Guarda Municipal do Rio de Janeiro,GM-RIO - Guarda Municipal do Rio de Janeiro,...,D,F,No prazo,Encerrado,Atendido,None,0,2022-12-01,POINT(-43.1691884 -22.962173),Reveillon
7227,18077988,2022-12-30 18:54:32,2022-12-30 22:14:21,35,2,65805,2,70,GM-RIO - Guarda Municipal do Rio de Janeiro,GM-RIO - Guarda Municipal do Rio de Janeiro,...,D,F,No prazo,Encerrado,Atendido,None,0,2022-12-01,POINT(-43.236608 -22.917704),Reveillon
7228,18077970,2022-12-30 18:43:04,2022-12-30 22:13:06,35,2,65805,2,70,GM-RIO - Guarda Municipal do Rio de Janeiro,GM-RIO - Guarda Municipal do Rio de Janeiro,...,D,F,No prazo,Encerrado,Atendido,None,0,2022-12-01,POINT(-43.236608 -22.917704),Reveillon
7231,18077984,2022-12-30 18:53:12,2022-12-30 22:13:46,35,2,65805,2,70,GM-RIO - Guarda Municipal do Rio de Janeiro,GM-RIO - Guarda Municipal do Rio de Janeiro,...,D,F,No prazo,Encerrado,Atendido,None,0,2022-12-01,POINT(-43.236608 -22.917704),Reveillon
7318,18078946,2022-12-31 11:52:48,2022-12-31 14:22:50,141,5,10629,640,70,GM-RIO - Guarda Municipal do Rio de Janeiro,GM-RIO - Guarda Municipal do Rio de Janeiro,...,D,F,No prazo,Encerrado,Atendido,None,0,2022-12-01,None,Reveillon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36479,18080335,2023-01-01 21:55:10,2023-01-02 02:30:13,64,3,81034,232,70,GM-RIO - Guarda Municipal do Rio de Janeiro,GM-RIO - Guarda Municipal do Rio de Janeiro,...,D,F,No prazo,Encerrado,Não atendido,None,0,2023-01-01,POINT(-43.2867915 -22.8912784),Reveillon
36480,18080337,2023-01-01 21:59:43,2023-01-02 02:24:31,64,3,81034,232,70,GM-RIO - Guarda Municipal do Rio de Janeiro,GM-RIO - Guarda Municipal do Rio de Janeiro,...,D,F,No prazo,Encerrado,Não atendido,None,0,2023-01-01,POINT(-43.2867915 -22.8912784),Reveillon
36481,18080341,2023-01-01 22:04:48,2023-01-02 02:18:39,64,3,81034,232,70,GM-RIO - Guarda Municipal do Rio de Janeiro,GM-RIO - Guarda Municipal do Rio de Janeiro,...,D,F,No prazo,Encerrado,Não atendido,None,0,2023-01-01,POINT(-43.2867915 -22.8912784),Reveillon
36488,18079926,2023-01-01 13:54:17,2023-01-01 21:24:33,42,3,19117,77,70,GM-RIO - Guarda Municipal do Rio de Janeiro,GM-RIO - Guarda Municipal do Rio de Janeiro,...,D,F,No prazo,Encerrado,Não atendido,None,2,2023-01-01,POINT(-43.2679728 -22.8477888),Reveillon


### Questão 8:
#### Quantos chamados desse subtipo foram abertos em cada evento?


In [87]:
df_5071 \
    .groupby('evento')['id_chamado'].nunique() \
    .reset_index(name='total_chamados') \
    .sort_values(by='total_chamados', ascending=False) 

,evento,total_chamados
2,Rock in Rio,834
0,Carnaval,241
1,Reveillon,137


### Questão 9:
#### Qual evento teve a maior média diária de chamados abertos desse subtipo?


In [112]:
df_merge = df_5071 \
    .groupby('evento')['id_chamado'].nunique() \
    .reset_index(name='total_chamados') \
    .merge(df_eventos.assign(duracao_evento=(df_eventos['data_final'] - df_eventos['data_inicial']).dt.days + 1)
                   .groupby('evento')['duracao_evento'].sum().reset_index(),
           on='evento', how='left')

df_merge['media_diaria_eventos'] = df_merge['total_chamados'] / df_merge['duracao_evento']
df_merge

,evento,total_chamados,duracao_evento,media_diaria_eventos
0,Carnaval,241,4,60.250000
1,Reveillon,137,3,45.666667
2,Rock in Rio,834,7,119.142857


### Questão 10:
#### Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos (Reveillon, Carnaval e Rock in Rio) e a média diária de chamados abertos desse subtipo considerando todo o período de 01/01/2022 até 31/12/2023.


In [113]:
df_merge['media_diaria_total'] = df_5071['id_chamado'].nunique() / (df_5071['data_inicio'].max() - df_5071['data_inicio'].min()).days
df_merge

,evento,total_chamados,duracao_evento,media_diaria_eventos,media_diaria_total
0,Carnaval,241,4,60.250000,59.22905
1,Reveillon,137,3,45.666667,59.22905
2,Rock in Rio,834,7,119.142857,59.22905
